# Member matching across tables: tops vs cosmos

## Stage 1: Jaro-Winkler: 1 chunk of tops vs full cosmos

Combo of criterions:
1. Exact mathcing criterion 
<br>
    zip code; dob_hashed; ssn_hashed; street number
  
2. Fuzzy match criterion
<br>
   Jaro_Winkle (Difflib: Retcliff/Obershelp?) member_first_name and member_last_name  
<br>
   (Please see the Appendix for background formula and initial demos for related algorithms)

## Another approach for step 3, let us search a member from tops from cosmos member tables on combo criteron with fuzzy match

## We decompose the task into several steps to allow the data to be loaded and processed with the currently available memory
All the cosmos members and part of tops (25 out of 55 chunks) are loaded back from saved csv file
<br>
Merging the tables using pandas merge function on keys
<br>
Then we load back the data as pandas dataframe and merge the tops and cosmos tables on keywords
<br>
The merged table will be saved as csv file
<br>
Fuzzy match will be performed based on the merged table in step 3


In [1]:
import pandas as pd
import os
import csv
import difflib
import jellyfish as jf
import itertools
from functools import reduce

from __future__ import division

In [2]:
# With modules imported defined, we start to work on the member tables
# defin ethe paths for dat aloading

data_dir = '/mapr/datalake/uhclake/perf/Sqoop_RX/ymalcow/501_mem_match_proj1/'

file_tops = 'tops10_mrg01.csv'
file_cosmos = 'cosmos_mrg01.csv'

path_tops = os.path.join(data_dir, file_tops)
path_cosmos = os.path.join(data_dir, file_cosmos)

# Define column names-table schema, and selected columns for table merge and member matching
mem_attrs = ['member_individual_identifier','src_cdb_xref_id','lgcy_src_id','social_security_number_hashed' , \
             'member_last_name','member_first_name' ,'member_middle_name','member_middle_initial_text', \
             'member_name_suffix' , 'mem_dob_hashed', 'memeber_zip_code_of_residence','member_sex','addr_line_1',\
             'addr_line_2','city_name','state','zipcode','country']
slt_attrs = ['social_security_number_hashed', 'mem_dob_hashed', 'member_last_name', 'member_first_name', \
             'zipcode', 'addr_line_1']

In [3]:
cosmos_mrg01 = pd.read_csv(path_cosmos, sep = ';', \
            quoting=csv.QUOTE_NONE, encoding='utf-8', error_bad_lines=False)


In [4]:
cosmos_mrg01.head()

,Unnamed: 0,social_security_number_hashed,mem_dob_hashed,member_last_name,member_first_name,zipcode,addr_line_1
0,0,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,0411cc37aeec0bc88cfe76834320671932c40eeb0230e0...,RICHARDSON,DANNY,36617.0,760 mollie court
1,1,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,8869b7bb249528e2ebb7ffc32563cf110661b44d01553f...,MCCLAIN,WILLIE,35208.0,1508 cullman ave
2,2,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,51fdf6d951baba49c81a74d6e92deaaf2dba676c405dde...,ROPER,ALBERT,36067.0,1816 buckeye drive
3,3,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,27de21fca3386878c880170adfb397069ea49f48d171ee...,SHAMBURGER,JOHN,36605.0,2056 height st
4,4,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,6b5a8ce077e17664730b5e010b320387783271fabfb0a0...,BATES,CLIFTON,36693.0,3012 blue ridge drive west


In [5]:
tops10_mrg01 = pd.read_csv(path_tops, sep = ';', \
            quoting=csv.QUOTE_NONE, encoding='utf-8', error_bad_lines=False)

In [6]:
tops10_mrg01.head()

,Unnamed: 0,social_security_number_hashed,mem_dob_hashed,member_last_name,member_first_name,zipcode,addr_line_1
0,0,552dc9e193e2982b016e0ada8321fa8b50c1923641836a...,32e2f6281b12f9fa438fffa47f6c3aca3fbb33b4019068...,BAIR,FORREST,21222,7629 old battle grove rd
1,1,f120bb5698d520c5691b6d603a00bfd662d13bf177a045...,ca4acbd7623f3b24850cc4738b4926cb9e98d884f6d611...,HULGAN,BRENDA,30281,55 caroline dr
2,2,223c210772e65dd2fb0c83e17b360246ee4ea9c99f9da6...,ad2ac68775efad36b871dc10daa7e31f5a7e9686f28c41...,BUSBY,SHIRLEY,39440,15 shady oak rd
3,3,aec3aff464c8fee20ebcbd3fb362acd88cc67bd2281a1e...,404356c321bcf2569b642b553ac25668648771068577f2...,BIRKENESS,RICHARD,64068,17803 n.e. 89th street
4,4,faa2f151591a8ceba7177092510f33488197aa9b16fe8d...,01611c51d8e9e36f35524403140115689b1e7dcd313092...,MCGEHAN,GREGORY,45066,420 thomas drive


In [10]:
tops10_mrg01.columns

Index(['Unnamed: 0', 'social_security_number_hashed', 'mem_dob_hashed',
       'member_last_name', 'member_first_name', 'zipcode', 'addr_line_1'],
      dtype='object')

In [9]:
tops10_mrg01.drop(columns = ['Unnamed: 0'], inplace = True)
#cosmos_mrg01.drop(columns = ['Unnamed: 0'], inplace = True)

In [10]:
tops10_mrg01.head()

,social_security_number_hashed,mem_dob_hashed,member_last_name,member_first_name,zipcode,addr_line_1
0,552dc9e193e2982b016e0ada8321fa8b50c1923641836a...,32e2f6281b12f9fa438fffa47f6c3aca3fbb33b4019068...,BAIR,FORREST,21222,7629 old battle grove rd
1,f120bb5698d520c5691b6d603a00bfd662d13bf177a045...,ca4acbd7623f3b24850cc4738b4926cb9e98d884f6d611...,HULGAN,BRENDA,30281,55 caroline dr
2,223c210772e65dd2fb0c83e17b360246ee4ea9c99f9da6...,ad2ac68775efad36b871dc10daa7e31f5a7e9686f28c41...,BUSBY,SHIRLEY,39440,15 shady oak rd
3,aec3aff464c8fee20ebcbd3fb362acd88cc67bd2281a1e...,404356c321bcf2569b642b553ac25668648771068577f2...,BIRKENESS,RICHARD,64068,17803 n.e. 89th street
4,faa2f151591a8ceba7177092510f33488197aa9b16fe8d...,01611c51d8e9e36f35524403140115689b1e7dcd313092...,MCGEHAN,GREGORY,45066,420 thomas drive


In [15]:
cosmos_mrg01.head()

,social_security_number_hashed,mem_dob_hashed,member_last_name,member_first_name,zipcode,addr_line_1
0,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,0411cc37aeec0bc88cfe76834320671932c40eeb0230e0...,RICHARDSON,DANNY,36617.0,760 mollie court
1,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,8869b7bb249528e2ebb7ffc32563cf110661b44d01553f...,MCCLAIN,WILLIE,35208.0,1508 cullman ave
2,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,51fdf6d951baba49c81a74d6e92deaaf2dba676c405dde...,ROPER,ALBERT,36067.0,1816 buckeye drive
3,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,27de21fca3386878c880170adfb397069ea49f48d171ee...,SHAMBURGER,JOHN,36605.0,2056 height st
4,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,6b5a8ce077e17664730b5e010b320387783271fabfb0a0...,BATES,CLIFTON,36693.0,3012 blue ridge drive west


In [11]:
# Extract the street number from member address
cosmos_mrg01['snum'] = cosmos_mrg01.addr_line_1.astype(str).str.split(' ', 1).apply(lambda x:x[0])
tops10_mrg01['snum'] = tops10_mrg01.addr_line_1.astype(str).str.split(' ', 1).apply(lambda x:x[0])

In [12]:
# Convert the cosmos zipcode from float to str
cosmos_mrg01['zipcode'] = cosmos_mrg01.zipcode.apply(lambda x : str(int(x)) if pd.notnull(x) else '')

In [13]:
%%time

# Merge the tables on blocking keys

CT_mrg02 = pd.merge(cosmos_mrg01, tops10_mrg01, left_on = ['zipcode', \
'social_security_number_hashed', 'mem_dob_hashed', 'snum'], right_on = ['zipcode', \
'social_security_number_hashed', 'mem_dob_hashed', 'snum'])

CPU times: user 1min 50s, sys: 3.07 s, total: 1min 53s
Wall time: 1min 53s


In [14]:
print ("There are {} records in merged cpsmos-tops table for possible member match".format(len(CT_mrg02.index)))
CT_mrg02.head()

There are 235848 records in merged cpsmos-tops table for possible member match


,social_security_number_hashed,mem_dob_hashed,member_last_name_x,member_first_name_x,zipcode,addr_line_1_x,snum,member_last_name_y,member_first_name_y,addr_line_1_y
0,9b041b5bc341fb9934a4e091216205aacd2b92be6ae697...,1360a295e3140c9c99b76f32828af83203d2f3378645b5...,DODGE,CHRISTOPHER,60103,550 sayer road,550,DODGE,CHRISTOPHER,550 sayer road
1,9b041b5bc341fb9934a4e091216205aacd2b92be6ae697...,1360a295e3140c9c99b76f32828af83203d2f3378645b5...,DODGE,CHRISTOPHER,60103,550 sayer rd,550,DODGE,CHRISTOPHER,550 sayer road
2,ae75aad75ec3e9cfa88f905fbf386b12b710275ca32793...,0070ce347fe003ee3c29735e3ebd3e69c6f20e88d8f72e...,JONES,JUDITH,27893,300a katharine ct,300a,JONES,JUDITH,300a katharine ct
3,a54c5417d05991f4a6de32bf1382b99871af3564e76df3...,08e04c43c4ac8bb16241088c7b272e795c129d93c9a7d1...,LYNCH,JAMES,90505,2634 w 230th pl,2634,LYNCH,JAMES,2634 230th place
4,8e06b3395bcca9946bad3e8bb6fabac45c757045aa9e0c...,eca12aa081594e51c7f1c340c63dc8eeb528e9ee19955d...,ETZEL,GEORGIA,92532,29315 third st,29315,ETZEL,G,29315 third st


In [15]:
CT_mrg02.tail()

,social_security_number_hashed,mem_dob_hashed,member_last_name_x,member_first_name_x,zipcode,addr_line_1_x,snum,member_last_name_y,member_first_name_y,addr_line_1_y
235843,8bb56c7df6fbd4d945db86b5bb51cff9467fe1ee746eb3...,44bb09e37d33f000d4fe3b2bb145896c7ede3c4f11160e...,HUNZIKER,CHRISTOPHER,98284,930 presidio pl,930,HUNZIKER,CHRISTOPHER,930 presidio pine
235844,29c4fa9ee48b2fb3a2c9c8e95e197efba4aa41be79490c...,f2be037d54abef4ce12edc7cf1e3ca8e893e6a8644669c...,DORTA,NEIDER,33014,7215 w 2nd ln,7215,DORTA,NEIDER,7215 w 2nd ln
235845,237260540f1baa17c41152a287755aebed41795b2e443f...,8f71bee9db79ac233f0e845f1b124692eac9d9e466c964...,WOART,DAVID,11727,9 grand canyon lane,9,WOART,DAVID,9 grand canyon lane
235846,0f6957652db07827c19d34fbad942dae8086c354580da0...,f14ad2567a32998dd188dc55f61fd39c77a09c97b45256...,ROBERTS-PRUYN,DIANA,30108,130 w college st,130,ROBERTS-PRUYN,DIANA,130 w. college st.
235847,e0c7048d208f4b613187a4ab23a1092ba5ad5f6351d636...,7ff4dab749140a3b38ac4ae8e09ac93356e76d2a1c20b7...,MELNARIK,MELISSA,54216,1309 river rd,1309,MELNARIK,MELISSA,1309 river rd


In [16]:
# Let us save this merged table to csv file first before going further with Jaro-Winkler analysis
# Save the merged tops-10chunks and all cosmos pandas table to csv file

out_tops = 'tops10cosmos_mrg02.csv'
path_out_CT10 = os.path.join(data_dir, out_tops)

CT_mrg02.to_csv(path_out_CT10, sep = ';')

In [17]:
CT_mrg02.dtypes

social_security_number_hashed    object
mem_dob_hashed                   object
member_last_name_x               object
member_first_name_x              object
zipcode                          object
addr_line_1_x                    object
snum                             object
member_last_name_y               object
member_first_name_y              object
addr_line_1_y                    object
dtype: object

In [18]:
CT_mrg02.iloc[128189]
CT_mrg02._slice(slice(128180,128189))

,social_security_number_hashed,mem_dob_hashed,member_last_name_x,member_first_name_x,zipcode,addr_line_1_x,snum,member_last_name_y,member_first_name_y,addr_line_1_y
128180,cf330406b698d3af04ef93021707572042539bdc4aa831...,560bf6e8405202daee728e478de7b38b993b6ff8bbb110...,PUCKHABER,JULIE,33067,4845 nw 89th ter,4845,PUCKHABER,JULIE,4845 nw 89th terrace
128181,cf330406b698d3af04ef93021707572042539bdc4aa831...,560bf6e8405202daee728e478de7b38b993b6ff8bbb110...,PUCKHABER,JULIE,33067,4845 nw 89th ter,4845,PUCKHABER,JULIE,4845 nw 89th terrace
128182,ba4797a9e33398e258d8bc0277154a34a8d13ee353e5eb...,433fb601e451c3d9f31842b4a5a856396adf02eba6c236...,MAURIZ,GERARDO,34219,2015 island estates drive,2015,MAURIZ,GERARDO,2015 island estates drive
128183,8798260510150a41875c02ff4e68f9a992675b12c54bb4...,88b98832763fb792be11e640314fe96f2c538d47a68a20...,HOWARD,MARGIE,31768,990 mcelroy rd,990,HOWARD,MARGIE,990 mcelroy rd
128184,8798260510150a41875c02ff4e68f9a992675b12c54bb4...,88b98832763fb792be11e640314fe96f2c538d47a68a20...,HOWARD,MARGIE,31768,990 mcelroy rd,990,HOWARD,MARGIE,990 mcelroy rd
128185,92af2d6923b35b4bbaa0e5badf11cfe65de23bca46c584...,6d550fc3960b0e9984f072351d995090b7588abd2af9de...,IANELLI,NICHOLAS,77389,6323 craigway rd,6323,IANELLI,NaN,6323 craigway rd
128186,5ac71d9bb85731d0b4ab34827ac2e0a6be783166c7ae5b...,3291adeb11894ada8e18791bbf355ca996ffebe8a2c060...,BUTTRAM,BARRIE,30236,8662 north shore drive,8662,BUTTRAM,BARRY,8662 north shore drive
128187,5ac71d9bb85731d0b4ab34827ac2e0a6be783166c7ae5b...,3291adeb11894ada8e18791bbf355ca996ffebe8a2c060...,BUTTRAM,BARRIE,30236,8662 north shore drive,8662,BUTTRAM,BARRY,8662 north shore drive
128188,d9f7d66af1c4391b00c2e9e98a8f6c3dabd361d729bb40...,675d3bcc6f74586682be8e3b0e95d2ee2b4db5af61b011...,ICANBERRY,JESSE,98146,2626 sw 112th st,2626,ICANBERRY,JESSE,2626 sw 112th st


In [19]:
# We see NaN for index 128185; so we need to process all the columns that to be used in similarity calculation
CT_mrg02['member_last_name_x'] = CT_mrg02.member_last_name_x.astype(str).apply(lambda x:str(x) if pd.notnull(x) else '')
CT_mrg02['member_last_name_y'] = CT_mrg02.member_last_name_y.astype(str).apply(lambda x:str(x) if pd.notnull(x) else '')
CT_mrg02['member_first_name_x'] = CT_mrg02.member_first_name_x.astype(str).apply(lambda x:str(x) if pd.notnull(x) else '')
CT_mrg02['member_first_name_y'] = CT_mrg02.member_first_name_y.astype(str).apply(lambda x:str(x) if pd.notnull(x) else '')


In [20]:
%%time
# Well, the table is saved, we do not have to worry about the dead kernel issue.
# Let us go ahead and move forward. We can run another job loading the saved table and continue the test at any point

# Calcute similarity score for last name

CT_mrg02['score1'] = CT_mrg02[['member_last_name_x','member_last_name_y',]].apply(lambda arr: jf.jaro_winkler(arr[0], arr[1]), axis = 1)
# Calcute similarity score for first name

CT_mrg02['score2'] = CT_mrg02[['member_first_name_x','member_first_name_y']].apply(lambda arr: jf.jaro_winkler(arr[0], arr[1]), axis = 1)


CPU times: user 28.3 s, sys: 4 ms, total: 28.4 s
Wall time: 28.3 s


In [21]:
display = ['member_last_name_x','member_last_name_y', 'member_first_name_x','member_first_name_y', 'score1', 'score2']
CT_mrg02[display].head()

,member_last_name_x,member_last_name_y,member_first_name_x,member_first_name_y,score1,score2
0,DODGE,DODGE,CHRISTOPHER,CHRISTOPHER,1.0,1.000000
1,DODGE,DODGE,CHRISTOPHER,CHRISTOPHER,1.0,1.000000
2,JONES,JONES,JUDITH,JUDITH,1.0,1.000000
3,LYNCH,LYNCH,JAMES,JAMES,1.0,1.000000
4,ETZEL,ETZEL,GEORGIA,G,1.0,0.714286


The above example shows that the Jaro-Winkler similarity between "Georgia" and "G" is only 0.714286, 
<br>
But "Georgia" and "G" may highly possible to be the first name of the same person, so we may consider
<br>
to introduce the the concept of "contains" into the combo criterion.

In [22]:
CT_mrg02[display]._slice(slice(128180,128189))

,member_last_name_x,member_last_name_y,member_first_name_x,member_first_name_y,score1,score2
128180,PUCKHABER,PUCKHABER,JULIE,JULIE,1.0,1.000000
128181,PUCKHABER,PUCKHABER,JULIE,JULIE,1.0,1.000000
128182,MAURIZ,MAURIZ,GERARDO,GERARDO,1.0,1.000000
128183,HOWARD,HOWARD,MARGIE,MARGIE,1.0,1.000000
128184,HOWARD,HOWARD,MARGIE,MARGIE,1.0,1.000000
128185,IANELLI,IANELLI,NICHOLAS,nan,1.0,0.000000
128186,BUTTRAM,BUTTRAM,BARRIE,BARRY,1.0,0.893333
128187,BUTTRAM,BUTTRAM,BARRIE,BARRY,1.0,0.893333
128188,ICANBERRY,ICANBERRY,JESSE,JESSE,1.0,1.000000


"BARRIE" vs "BARRY" : 0.893333, hmmmmmmmm...... interesting!

In [23]:
# Let us filter for some cases that we may be more interested in generally

CT_mrg02[CT_mrg02.score1 > 0.96][CT_mrg02.score1 <1][display].sort_values('score1', ascending = True).head(20)

/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


,member_last_name_x,member_last_name_y,member_first_name_x,member_first_name_y,score1,score2
200491,BETANCOURT,BENTACOURT,ALEXANDRA,ALEXANDRA,0.960000,1.0
17113,TREJO MICHAELANGEL,TREJO MICHELANGELI,ANDREINA,ANDREINA,0.960131,1.0
214323,SIMS PERKINS,SIMO PERKINS,CARMEN,CARMEN,0.961111,1.0
181696,DIXON,DIXSON,DOROTHY,DOROTHY,0.961111,1.0
190495,LAJOUD,LAJUD,JORGE,JORGE,0.961111,1.0
14545,DVOARK,DVORAK,BLAKE,BLAKE,0.961111,1.0
26147,SINTEK,SINEK,BRITTANY,BRITTANY,0.961111,1.0
26146,SINTEK,SINEK,BRITTANY,BRITTANY,0.961111,1.0
26145,SINTEK,SINEK,BRITTANY,BRITTANY,0.961111,1.0
26144,SINTEK,SINEK,BRITTANY,BRITTANY,0.961111,1.0


In [25]:
CT_mrg02[CT_mrg02.score2 > 0.96][CT_mrg02.score2 <1][display].sort_values('score2', ascending = True).head(20)

/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,member_last_name_x,member_last_name_y,member_first_name_x,member_first_name_y,score1,score2
80535,GOODLOE,GOODLOE,ABA-NYAMEKYE,ABA NYAMEKYE,1.000000,0.961111
209213,VILLASOSA,VILLASOSA,ABBIS,ABBLIS,1.000000,0.961111
206983,WELLNER,WELLNER,MEGHAN,MEGAN,1.000000,0.961111
173256,JOSEPH,JOSEPH,VENGDY,VENDGY,1.000000,0.961111
88069,MACK,MACK,LOUISE,LOUSIE,1.000000,0.961111
88070,MACK,MACK,LOUISE,LOUSIE,1.000000,0.961111
175683,HAMPTON,HAMPTON,SHELIA,SHEILA,1.000000,0.961111
189721,MOREJON,MOREJON,MARCIA,MARIA,1.000000,0.961111
47767,WILLIAMS,WILLIAMS,JASYLN,JASLYN,1.000000,0.961111
188439,MANN,MANN,JACLYN,JACYN,1.000000,0.961111


In [54]:
# Let us filter for some cases that we may be more interested in generally

CT_mrg02[CT_mrg02.score1 > 0.9][CT_mrg02.score1 <1][display].sort_values('score1', ascending = True).head(20)

/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


,member_last_name_x,member_last_name_y,member_first_name_x,member_first_name_y,score1,score2
146395,SCHLEHUBER,SCHILHUBER,CATHY,CATHY,0.901481,1.0
187546,DE-LA-MONTANYA,DE LA MONTANY,HILDA,HILDA,0.901832,1.0
187547,DE-LA-MONTANYA,DE LA MONTANY,HILDA,HILDA,0.901832,1.0
224233,ORTIZ,ORITIZ,GERALD,GERALD,0.902222,1.0
109674,REYES,REEYES,SUSAN,SUSAN,0.902222,1.0
169986,REYES,REEYES,SALVADOR,SALVADOR,0.902222,1.0
201078,RIVERAJR,RIVERA RICO,JOSE,JOSE,0.902273,1.0
64633,PYLATIUK,PLYLATILUK,GREG,GREG,0.902500,1.0
140058,VAUGHAN,VAUNGHAN,LARRY,LARRY,0.904167,1.0
206942,TILLMAN,PILLMAN,WILLIAM,WILLIAM,0.904762,1.0


In [55]:
CT_mrg02[CT_mrg02.score2 > 0.9][CT_mrg02.score2 <1][display].sort_values('score2', ascending = True).head(20)

/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,member_last_name_x,member_last_name_y,member_first_name_x,member_first_name_y,score1,score2
26835,ANDREWS,ANDREWS,JULIE,JUILIE,1.0,0.902222
34179,BROWN,BROWN,WANY A,WAYNA,1.0,0.902222
26834,ANDREWS,ANDREWS,JULIE,JUILIE,1.0,0.902222
26833,ANDREWS,ANDREWS,JULIE,JUILIE,1.0,0.902222
59923,HOWARD,HOWARD,ISAIAH,ISIAH,1.0,0.902222
59924,HOWARD,HOWARD,ISAIAH,ISIAH,1.0,0.902222
21848,BISHOP,BISHOP,ISAIAH,ISIAH,1.0,0.902222
112735,ROBLIN,ROBLIN,MEAGAN,MEGAN,1.0,0.902222
112736,ROBLIN,ROBLIN,MEAGAN,MEGAN,1.0,0.902222
226797,PHAM,PHAM,QUOCANANTHO,QUOC ANH,1.0,0.902273


## For each member of Tops, matching it with the cosmos members to see if there is the same member

In [ ]:
# We see NaN for index 128185; so we need to process all the columns that to be used in similarity calculation
CT_mrg02['member_last_name_x'] = CT_mrg02.member_last_name_x.astype(str).apply(lambda x:str(x) if pd.notnull(x) else '')
CT_mrg02['member_last_name_y'] = CT_mrg02.member_last_name_y.astype(str).apply(lambda x:str(x) if pd.notnull(x) else '')
CT_mrg02['member_first_name_x'] = CT_mrg02.member_first_name_x.astype(str).apply(lambda x:str(x) if pd.notnull(x) else '')
CT_mrg02['member_first_name_y'] = CT_mrg02.member_first_name_y.astype(str).apply(lambda x:str(x) if pd.notnull(x) else '')

In [ ]:
# Merge the tables on blocking keys

CT_mrg02 = pd.merge(cosmos_mrg01, tops10_mrg01, left_on = ['zipcode', \
'social_security_number_hashed', 'mem_dob_hashed', 'snum'], right_on = ['zipcode', \
'social_security_number_hashed', 'mem_dob_hashed', 'snum'])

In [26]:
def clean_names(df_name):
    df_name['member_last_name'] = df_name.member_last_name.astype(str).apply(lambda x:str(x) if pd.notnull(x) else '')
    df_name['member_first_name'] = df_name.member_first_name.astype(str).apply(lambda x:str(x) if pd.notnull(x) else '')   

In [27]:
clean_names(cosmos_mrg01)

In [28]:
clean_names(tops10_mrg01)

In [29]:
cosmos_mrg01.head()

,social_security_number_hashed,mem_dob_hashed,member_last_name,member_first_name,zipcode,addr_line_1,snum
0,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,0411cc37aeec0bc88cfe76834320671932c40eeb0230e0...,RICHARDSON,DANNY,36617,760 mollie court,760
1,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,8869b7bb249528e2ebb7ffc32563cf110661b44d01553f...,MCCLAIN,WILLIE,35208,1508 cullman ave,1508
2,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,51fdf6d951baba49c81a74d6e92deaaf2dba676c405dde...,ROPER,ALBERT,36067,1816 buckeye drive,1816
3,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,27de21fca3386878c880170adfb397069ea49f48d171ee...,SHAMBURGER,JOHN,36605,2056 height st,2056
4,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,6b5a8ce077e17664730b5e010b320387783271fabfb0a0...,BATES,CLIFTON,36693,3012 blue ridge drive west,3012


In [30]:
tops10_mrg01.head()

,social_security_number_hashed,mem_dob_hashed,member_last_name,member_first_name,zipcode,addr_line_1,snum
0,552dc9e193e2982b016e0ada8321fa8b50c1923641836a...,32e2f6281b12f9fa438fffa47f6c3aca3fbb33b4019068...,BAIR,FORREST,21222,7629 old battle grove rd,7629
1,f120bb5698d520c5691b6d603a00bfd662d13bf177a045...,ca4acbd7623f3b24850cc4738b4926cb9e98d884f6d611...,HULGAN,BRENDA,30281,55 caroline dr,55
2,223c210772e65dd2fb0c83e17b360246ee4ea9c99f9da6...,ad2ac68775efad36b871dc10daa7e31f5a7e9686f28c41...,BUSBY,SHIRLEY,39440,15 shady oak rd,15
3,aec3aff464c8fee20ebcbd3fb362acd88cc67bd2281a1e...,404356c321bcf2569b642b553ac25668648771068577f2...,BIRKENESS,RICHARD,64068,17803 n.e. 89th street,17803
4,faa2f151591a8ceba7177092510f33488197aa9b16fe8d...,01611c51d8e9e36f35524403140115689b1e7dcd313092...,MCGEHAN,GREGORY,45066,420 thomas drive,420


In [41]:
def criteron1(s1, s2):
    if s1['social_security_number_hashed'] != s2['social_security_number_hashed']:
        return False
    if s1['mem_dob_hashed'] != s2['mem_dob_hashed']:
        return False
    
    #ln_score = jf.jaro_winkler(s1['member_last_name'], s2['member_last_name'])
    #fn_score = jf.jaro_winkler(s1['member_first_name'], s2['member_first_name'])
    if jf.jaro_winkler(s1['member_last_name'], s2['member_last_name']) < 0.96:
        return False
    if jf.jaro_winkler(s1['member_first_name'], s2['member_first_name']) < 0.96:
        return False
    return True   

In [42]:
# will add one more parameter threshhold to make the current 0.96 coefficient tuneable
def criteron2(s1, s2):
    
    if s1['mem_dob_hashed'] != s2['mem_dob_hashed']:
        return False
    
    #ln_score = jf.jaro_winkler(s1['member_last_name'], s2['member_last_name'])
    #fn_score = jf.jaro_winkler(s1['member_first_name'], s2['member_first_name'])
    
    if jf.jaro_winkler(s1['member_last_name'], s2['member_last_name']) < 0.96:
        return False
    if jf.jaro_winkler(s1['member_first_name'], s2['member_first_name']) < 0.96:
        return False
    
    if s1['zipcode'] != s2['zipcode']:
        return False
    return True 

In [39]:
type(tops10_mrg01.iloc[4])

pandas.core.series.Series

In [45]:
for sr2 in cosmos_mrg01.head():
    print (type(sr2))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [ ]:
all_match = {}
for i in range(100):
    list_match = []
    sr1 = tops10_mrg01.iloc[i]
    print (type(sr1))
    for j in range(len(cosmos_mrg01.index)):
        curr_match = False
        sr2 = cosmos_mrg01.iloc[j]
        if criteron1(sr1, sr2):
            curr_match = True
            list_match.append(sr2)
        elif criteron2(sr1, sr2):
            curr_match = True
            list_match.append(sr2)
    all_match[sr1] = sr2     
        
        

<class 'pandas.core.series.Series'>


# To be continued......

#  *********************************************************************

## Appendix: The above application is based on the following premium experimentals:
Pardon on the disporganized raw experiments, plz

## Fuzzy string match - method one Python-difflib, and jellyfish:

1. Load the member data into python pandas 
2. Build up the data structurte for matching algorithms
3. Apply difflib to the tables/attribute, which is based on Retcliff/Obershelp Algorithms
4. Investigate a second solution of jellyfish, which is based on Jaro/Wilkler Algorithms

# Retcliff/Obershelp vs Jaro-Winkler
## 1. Retcliff/Obershelp
$Dr = 2*Km/(|S1| + |S2|)$, 
<br>
$Km$: number of matching characters based on Anchor; |S1|, |S2| length of S1 and S2
<br>
Anchor: longest substring such that S1 and S2 have in common

## 2. Jaro Distance (1989)
$Dj = (1/3 ) * (m/|S1| + m/|S2| + (m - t)/m)$
<br>$
$m$ : number of matching symbols;
<br>
$t$ : number of transpositions



## Now in this test we load the csv files with delimitor ';' instead of ','
originally, the csv file was seporated with delimitor ',', which brings significant issue, since some attributes come with ',' in the string. So after reading back delimited by ',', some record got more features than it supposed to be:

$ select count(1) from cosmos_mem where addr_line_1 like "%,%";

OK
18253

So the table is extracted to csv file using ';' as the delimitor instead of ',':
For examples:
INSERT OVERWRITE LOCAL DIRECTORY '/mapr/datalake/uhclake/perf/Sqoop_RX/ymalcow/501_mem_match_proj1/tops_all2_csv' ROW FORMAT DELIMITED FIELDS TERMINATED BY '\;' select * from tops_mem;

INSERT OVERWRITE LOCAL DIRECTORY '/mapr/datalake/uhclake/perf/Sqoop_RX/ymalcow/101_clustering_proj01/prep01ML_all2_csv' ROW FORMAT DELIMITED FIELDS TERMINATED BY '\;' select * from merged_cosmos_dr_claim_4ml_t01;



In [ ]:
import pandas as pd
import os
import csv
import difflib
import jellyfish
import itertools

from __future__ import division

In [ ]:
def jaccard_similarity(a, b):
    x = set(a)
    y = set(b)
    return len(x & y) / len(x | y)

In [ ]:
# Define jaccard distance based on jaccard similarity
def jaccard_distance(a, b): return 1 - jaccard_similarity(a, b)

In [ ]:
# With modules imported defined, we start to work on the member tables

In [ ]:
data_dir = '/mapr/datalake/uhclake/perf/Sqoop_RX/ymalcow/501_mem_match_proj1/'
dir_tops = 'tops_all2_csv'
dir_cosmos = 'cosmos_all2_csv'

path_tops = os.path.join(data_dir, dir_tops)
path_cosmos = os.path.join(data_dir, dir_cosmos)

In [ ]:
print (os.getcwd())

In [ ]:
topses = os.listdir(path_tops)
cosmoses = os.listdir(path_cosmos)

In [ ]:
mem_attrs = ['member_individual_identifier','src_cdb_xref_id','lgcy_src_id','social_security_number_hashed' , 'member_last_name','member_first_name' ,'member_middle_name','member_middle_initial_text', 'member_name_suffix' , 'mem_dob_hashed', 'memeber_zip_code_of_residence','member_sex','addr_line_1','addr_line_2','city_name','state','zipcode','country']

In [ ]:
len(mem_attrs)

In [ ]:
print ("No. of cosmos file vs. No. of tops files are {} : {}".format(len(cosmoses), len(topses)))

In [ ]:
# Now let us traverswe through all the cosmos tables to see if thre is better mtches
# First we need to load all the files for tops member, 
# Extract the column of adrress line 1 and concat to the dataframe

addr1_tops = []
cnt = 0
cnt_in = 0

for tops in topses:
    cnt += 1
    if tops[-3:] == 'crc':
        continue
    cnt_in += 1
    curr_tops = pd.read_csv(os.path.join(path_tops, tops), sep = ';', names = mem_attrs, quoting=csv.QUOTE_NONE, encoding='utf-8', error_bad_lines=False)

    # convert every character to lower case
    curr_lower_tops = list(set(curr_tops.addr_line_1.apply(lambda x : str(x).lower() if pd.notnull(x) else '')))
    print ("{} out of {}. File name={}; No. of curr uniq addr is {}".format(cnt_in, cnt, tops, len(curr_lower_tops)))
    addr1_tops.extend(curr_lower_tops)

In [ ]:
# Use only one chunk for each type of claim to perform initial test
samp_tops = pd.read_csv(os.path.join(path_tops, topses[1]), sep = ';', names = mem_attrs, error_bad_lines=False)
samp_cosmos = pd.read_csv(os.path.join(path_cosmos, cosmoses[1]), sep = ';', names = mem_attrs, error_bad_lines=False)

lower_chk01_cosmos = samp_cosmos.addr_line_1.apply(lambda x : str(x).lower() if pd.notnull(x) else '')
lower_chk01_tops = samp_tops.addr_line_1.apply(lambda x : str(x).lower() if pd.notnull(x) else '')

In [ ]:
len(lower_chk01_cosmos)

In [ ]:
%%time

# Calculate the best 10 ratios of similarity between the first three form cosmos vs all tops
cnt = 0
ratio_all = []

for i in range(20000, 20002):
         
    curr_sim = difflib.get_close_matches(lower_chk01_cosmos[i], addr1_tops, 10)
    print ("{}. for '{}',there are {} close matches:\n{}".format(i, lower_chk01_cosmos[i], len(curr_sim), curr_sim))
    ratio = []
    sm = difflib.SequenceMatcher()
    sm.set_seq1(lower_chk01_cosmos[i])
    for s in curr_sim:
        sm.set_seq2(s)
        ratio.append(sm.ratio())
    ratio_all.append(ratio)

In [ ]:
#There are some duplications observed from the outputs above, let us make the list unique
addr1_tops = list(set(addr1_tops))

In [ ]:
%%time

# Calculate the best 10 ratios of similarity between the first three form cosmos vs all tops
cnt = 0
ratio_all = []

for i in range(20000, 20002):
         
    curr_sim = difflib.get_close_matches(lower_chk01_cosmos[i], addr1_tops, 10, 0.6)
    print ("The address line 1 for tops are unique now.\n{}. for '{}',there are {} close matches:\n{}".format(i, lower_chk01_cosmos[i], len(curr_sim), curr_sim))
    ratio = []
    sm = difflib.SequenceMatcher()
    sm.set_seq1(lower_chk01_cosmos[i])
    for s in curr_sim:
        sm.set_seq2(s)
        ratio.append(sm.ratio())
    ratio_all.append(ratio)

In [ ]:
ratio_all

In [ ]:
# Now let us use the results above and do some demo to compare different algorithms by this chance
# First setup the demo data as a list of addresses:
addr20001 = ['2403 battlefield parkway', '3749 battlefield parkway', '2403 campfield parkway', \
             '23 canterfield parkway', '801 battlefield pkwy', '21319 butterfield parkway', '14377 butterfield parkway', \
             '14358 butterfield parkway', '20403 fairfield parkway', '7912 battlefield park']

similarities = [(n1,n2, jellyfish.levenshtein_distance(n1,n2), jellyfish.damerau_levenshtein_distance(n1,n2), \
                 jellyfish.jaro_distance(n1,n2), jellyfish.jaro_winkler(n1,n2),\
                 jaccard_similarity(n1,n2)) for n1 in addr20001 for n2 in addr20001]


In [ ]:
comp = pd.DataFrame(similarities)
comp.columns = ['n1', 'n2', 'Levenshtein', 'Damerau_Levenshtein', 'Jaro', 'Jaro_Winker', 'Jaccard']
comp

In information theory and computer science, the Damerau–Levenshtein distance (named after Frederick J. Damerau and Vladimir I. Levenshtein)[citation needed] is a "distance" (string metric) between two strings, i.e., finite sequence of symbols, given by counting the minimum number of operations needed to transform one string into the other, where an operation is defined as an insertion, deletion, or substitution of a single character, or a transposition of two adjacent characters.
Take for example the edit distance between CA and ABC. The Damerau–Levenshtein distance LD(CA,ABC) = 2 because CA -> AC -> ABC, but the optimal string alignment distance OSA(CA,ABC) = 3 because if the operation CA -> AC is used, it is not possible to use AC -> ABC because that would require the substring to be edited more than once, which is not allowed in OSA, and therefore the shortest sequence of operations is CA -> A -> AB -> ABC. Note that for the optimal string alignment distance, the triangle inequality does not hold: OSA(CA,AC) + OSA(AC,ABC) < OSA(CA,ABC), and so it is not a true metric.

80-96%

first 3 of first name + first 3 of last name and first five of (addr)/zip

# How to make the implementation feasible with dozens of millions of records?
##     -Let us investigate the possible solution using blocking

It is not feasible to do cross comparison by fuzzy searching each member record of one table in the otyher tops table, approximately 20 million X 20 million level computation! Blocking is an effective method to reduce the number of recod pais that need to be compared. Here I dtart from two options:
1. Grouping by zip code
    Acoording to wikipedia.org there are over 42,000 ZIP codes in US:
    https://en.wikipedia.org/wiki/ZIP_Code
2. using soundex function from the python library jellyfish:
    Soundex is a code that is supposed to indicate strings that begin with similar sounds

In [ ]:
samp_tops.head()
samp_cosmos.head()

In [ ]:
samp_tops.head()

In [ ]:
samp_cosmos.rename(columns = {'addr_line_1': 'addr_line_1C'}, inplace = True)

In [ ]:
# Demo on soundex using the sample chunk of cosmos members and tops members
samp_cosmos['sndx'] = samp_cosmos.addr_line_1C.apply(jellyfish.soundex)
# samp_tops['sndx'] = samp_tops.addr_line_1.apply(jellyfish.soundex)


In [ ]:
samp_tops['addr_line_1'] = samp_tops.addr_line_1.apply(lambda x : str(x) if pd.notnull(x) else '')
samp_tops['sndx'] = samp_tops.addr_line_1.apply(jellyfish.soundex)

In [ ]:
sc_cnt = samp_cosmos['sndx'].value_counts()
print (type(sc_cnt), sc_cnt.head())
sc_cnt[sc_cnt == 10]

In [ ]:
samp_cosmos.columns

In [ ]:
samp_tops.columns

In [ ]:
 samp_tops[samp_tops.sndx == '4550'].head()

In [ ]:
s1 = samp_tops[samp_tops.sndx == '4550'][['addr_line_1', 'sndx']]
#s2 = samp_cosmos[samp_cosmos.sndx == '4550']['addr_line_1C', 'sndx']

samp_tops[samp_tops.sndx == '4550'].merge(samp_cosmos[samp_cosmos.sndx == '4550'], on = 'sndx')[['addr_line_1C', 'addr_line_1', 'sndx']]

In [ ]:
print ("Cosmos sample chunk got {} number of soundex out of total {} records; \ n And the value counts for each soundex are:\n{}".format(samp_cosmos['sndx'].nunique(),samp_cosmos.shape[0],sc_cnt))

In [ ]:
samp_tops.head()

In [ ]:
samp_tops[samp_tops.sndx == '1235'].count()

In [ ]:
samp_cosmos[samp_cosmos.sndx == '1235']

In [ ]:
s1_tops = samp_tops[samp_tops.sndx == '1235']['addr_line_1C']

In [ ]:
samp_tops[samp_tops.sndx == '1235'].merge(samp_cosmos[samp_cosmos.sndx == '1235'], on = 'sndx')['addr_line_1C', 'addr_line_1']

In [ ]:
samp_tops[samp_tops.sndx == '1235'].merge(samp_cosmos[samp_cosmos.sndx == '1235'], on = 'sndx')['addr_line_1C', 'addr_line_1']

In [ ]:
samp_tops[samp_tops.sndx == 'P120']

In [ ]:
samp_cosmos[samp_cosmos.sndx == 'P120']

In [ ]:
samp_tops[samp_tops.sndx == 'p120'].merge(samp_cosmos[samp_cosmos.sndx == 'P120'], on = 'sndx')['addr_line_1C', 'addr_line_1']

1. select the ssn with the max number of counts from cosmos_mem

select c.ssn, c.counter from (select d.social_security_number_hashed as ssn, count(*) as counter from cosmos_mem d group by d.social_security_number_hashed) c join (select max(f.counter) as cntMax from (select mem.social_security_number_hashed, count(*) as counter from cosmos_mem mem group by mem.social_security_number_hashed)f)g where c.counter = g.cntMax;
OK
e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855	4905202

Duplications found. Let us keep confirm with this ssn:

select * from cosmos_mem where social_security_number_hashed == 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855' limit 5;

OK
62495351	426814555	BHM0217190495722500	e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855	JOHNSON	SAMMIE	L	L		a2176455ad29d9382d9e61aaff6f7021f414c93d12806f9ce4bcb3fa89623bb5	36869	MALE	218 16TH AVE S		PHENIX CITY	AL	36869	
465055251	611244034	BHM0219890336028900	e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855	WOODS	WILLIE	J	J		9af28c4efccc06efb6f41fa29f0e4c79c36af7f762d39c05f6d3d9d8d664dfc3	36108	MALE	3835 RIVERIA RD.		MONTGOMERY	AL	36108	
95549897	571128636	BHM0219891289762000	e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855	WASHINGTON	ARLANDERS	L	L	a311fe6b539bcfb76e526915845ab0413841c7d41b109cca7c7d36aed4d5eada	36092	MALE	604 MARSHALL ST		WETUMPKA	AL	36092	
481025873	623134285	BHM0221195769134600	e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855	BENARD	ROBERT	E	E		525c6164ddefe07400c5ddf6974b2f17cf3b222173405bb3a0b30c59badfa0c4	36083	MALE	655 REC CENTER RD		TUSKEGEE	AL	36083	
61409303	558502262	BHM0223990110540300	e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855	ENGLISH	STEPHEN	S	S		4b100e8b355e21204c3ce99c7cea6d3eaf1d6e144cd30ddf676b9c4f48e997f7	35214	MALE	1705 CHELSEA AVE		BIRMINGHAM	AL	35214	
Time taken: 26.052 seconds, Fetched: 5 row(s)

Yes, it is confirmed that, then
1' How about tops? :
Query result:
OK
f120bb5698d520c5691b6d603a00bfd662d13bf177a04571f9d10c0745dfa2a5	8511228


2. How mash hashed ssn are duplicated? :

Cosmos:

tops:
select social_security_number_hashed as ssn, count(*) as counter from tops_mem group by social_security_number_hashed having counter > 1;
OK
...
fffcf724fdb3de9ef29f3c1061048b952cbc4c4cca8a4ad2c4c82f675e9ca1a7	2
fffd0eb99dc558dd6215e7776dd1e48c811f1f202d0c93ef9200f322b97bf3ed	2
fffd7877d0b10106935ab61c4920b67bb88bb1736d64228ddbb66cd255e1708f	2
fffe535859992730c984ced6661f5a7fb3c9691407876ae779ea5bdb46120ed5	2
ffff9990155834c2f45412d970b96fe287fffbbf4f732fbb70634d613741d720	2
ffffad5176879d166e05ff89182726cba703d3246810f7cdc8fd4bb63b58c792	2
ffffb18d4b89956723cb70fde6d20c82f22b9189d93dc99cae8b526d131f02e4	2
ffffe4d2edbea493550b0ef874cb7fe2820ce17678d675aeb3021e001b9c84fd	2
ffffeecd55730a30b469835e546da7e204c3ff4095aad0298d4701247f2c3184	2
fffff55fffc3eb82e2252ef66b5cd8a1b3090bab422fb74e18b9d3f8f47440f5	2
Time taken: 55.799 seconds, Fetched: 10237568 row(s)

In [ ]:
samp_tops.merge(samp_cosmos, on = 'sndx')

In [ ]:
print ("test kernel")

# Now for some complete member matchs
## Single chunk based on the management table partition will be used for initial test

Combo of criterions:
1. Exact mathcing criterion
   zip code; dob_hashed; ssn_hashed
2. Fuzzy match criterion
   Jaro_Winkle (Difflib: Retcliff/Obershelp?) member_first_name and member_last_name  

In [ ]:
samp_tops.head()

In [ ]:
samp_cosmos.head()

In [ ]:
samp_cosmos['snum_c'] = samp_cosmos.addr_line_1C.str.split(' ', 1).apply(lambda x:x[0])
samp_tops['snum_t'] = samp_tops.addr_line_1.str.split(' ', 1).apply(lambda x:x[0])

In [ ]:
slt_cosmos = samp_cosmos[['social_security_number_hashed', 'mem_dob_hashed', 'member_last_name', 'member_first_name', 'zipcode', 'snum_c']]
slt_tops = samp_tops[['social_security_number_hashed', 'mem_dob_hashed', 'member_last_name', 'member_first_name', 'zipcode', 'snum_t']]

In [ ]:
slt_cosmos.head()

In [ ]:
slt_tops.head()

In [ ]:
slt_mrg = pd.merge(slt_cosmos, slt_tops, left_on = ['zipcode', 
'social_security_number_hashed', 'mem_dob_hashed', 'snum_c'], right_on = ['zipcode', 
'social_security_number_hashed', 'mem_dob_hashed', 'snum_t'])

In [ ]:
slt_mrg.head()

## Now it is time to move to the full tables and apply Jaro-Winkle, since the matching between the chunks does not generate interesting pairs.
How many records in cosmos and tops member tables respectly?
 select count(1) from cosmos_mem;
 OK
27126473

select count(1) from tops_mem;
OK
82572107

So let us start to search the first chunk of tops from full cosmos, since we need to see faster results at this initial stage

In [ ]:
# Use only one chunk for each type of claim to perform initial test
samp_tops = pd.read_csv(os.path.join(path_tops, topses[1]), sep = ';', names = mem_attrs, error_bad_lines=False)
samp_cosmos = pd.read_csv(os.path.join(path_cosmos, cosmoses[1]), sep = ';', names = mem_attrs, error_bad_lines=False)

In [ ]:
samp_tops.member_first_name.count()

In [ ]:
samp_tops.head()

In [ ]:
lower_chk01_cosmos = samp_cosmos.addr_line_1.apply(lambda x : str(x).lower() if pd.notnull(x) else '')
lower_chk01_tops = samp_tops.addr_line_1.apply(lambda x : str(x).lower() if pd.notnull(x) else '')

In [ ]:
import difflib

In [ ]:
# First one that starts to work!!!
print(difflib.get_close_matches('262 plantation cir lot 21', lower_chk01_cosmos))

In [ ]:
print(difflib.get_close_matches('262 plantation cir lot 21', lower_chk01_tops))

In [ ]:
cnt = 0
ratio_all = []

for address1 in lower_chk01_cosmos:
    cnt += 1
    print ("the currnt count is {}".format(cnt))
    if cnt > 3:
        break
    
    curr_sim = difflib.get_close_matches(address1, lower_chk01_tops, 5)
    print ("{}. for '{}',there are {} close matches:\n{}".format(cnt, address1, len(curr_sim), curr_sim))
    ratio = []
    sm = difflib.SequenceMatcher()
    sm.set_seq1(address1)
    for s in curr_sim:
        sm.set_seq2(s)
        ratio.append(sm.ratio())
    ratio_all.append(ratio)

In [ ]:
print ("No. of cosmos file vs. No. of tops files are {} : {}".format(len(cosmoses), len(topses)))

In [ ]:
topses

In [ ]:
# Now let us traverswe through all the cosmos tables to see if thre is better mtches
# First we need to load all the files for tops member, 
#       extract the column of adrress line 1 and concat to the dataframe

addr1_all = []
cnt = 0
cnt_in = 0

for tops in topses:
    cnt += 1
    if tops[-3:] == 'crc':
        continue
    cnt_in += 1
    curr_tops = pd.read_csv(os.path.join(path_tops, tops), sep = ';', names = mem_attrs, quoting=csv.QUOTE_NONE, encoding='utf-8', error_bad_lines=False)

    # convert every character to lower case
    curr_lower_tops = list(set(curr_tops.addr_line_1.apply(lambda x : str(x).lower() if pd.notnull(x) else '')))
    print ("{} out of {}. File name={}; No. of curr uniq addr is {}".format(cnt_in, cnt, tops, len(curr_lower_tops)))
    addr1_all.extend(curr_lower_tops)

In [ ]:
topses[0][-3:]

In [ ]:
# Now let us traverswe through all the cosmos tables to see if thre is better mtches
# First we need to load all the files for tops member, 
#       extract the column of adrress line 1 and concat to the dataframe

addr1_all = []
cnt = 0
cnt_in = 0

for tops in topses:
    cnt += 1
    if tops[-3:] == 'crc':
        continue
    cnt_in += 1
    curr_tops = pd.read_csv(os.path.join(path_tops, tops), sep = ',', names = mem_attrs, quoting=csv.QUOTE_NONE, encoding='utf-8', error_bad_lines=False)

    # convert every character to lower case
    curr_lower_tops = set(curr_tops.addr_line_1.apply(lambda x : str(x).lower() if pd.notnull(x) else ''))
    print ("{} out of {}. File name={}; No. of curr uniq addr is {}".format(cnt_in, cnt, tops, len(curr_lower_tops)))
    addr1_all.extend(curr_lower_tops)

In [ ]:
# Now let us traverswe through all the cosmos tables to see if thre is better mtches
# First we need to load all the files for tops member, 
#       extract the column of adrress line 1 and concat to the dataframe

addr1_all = []
cnt = 0
cnt_i = 0

for tops in topses:
    cnt += 1
    if tops[-3:] == 'crc':
        continue
    cnt_in += 1
    curr_tops = pd.read_csv(os.path.join(path_tops, tops), sep = ',', names = mem_attrs, quoting=csv.QUOTE_NONE, encoding='utf-8', error_bad_lines=False)

    # convert every character to lower case
    curr_lower_tops = set(curr_tops.addr_line_1.apply(lambda x : str(x).lower() if pd.notnull(x) else ''))
    print ("{} out of {}. File name={}; No. of curr uniq addr is {}".format(cnt_in, cnt, tops, len(curr_lower_tops)))
    addr1_all.extend(curr_lower_tops)

In [ ]:
# Now let us traverswe through all the cosmos tables to see if thre is better mtches
# First we need to load all the files for tops member, 
#       extract the column of adrress line 1 and concat to the dataframe
import csv

addr1_all = []
cnt = 0
cnt_in = 0

for tops in topses:
    cnt += 1
    if tops[-3:] == 'crc':
        continue
    cnt_in += 1
    curr_tops = pd.read_csv(os.path.join(path_tops, tops), sep = ',', names = mem_attrs, quoting=csv.QUOTE_NONE, encoding='utf-8', error_bad_lines=False)

    # convert every character to lower case
    curr_lower_tops = set(curr_tops.addr_line_1.apply(lambda x : str(x).lower() if pd.notnull(x) else ''))
    print ("{} out of {}. File name={}; No. of curr uniq addr is {}".format(cnt_in, cnt, tops, len(curr_lower_tops)))
    addr1_all.extend(curr_lower_tops)

In [ ]:
len(addr1_all)

In [ ]:
# Calculate the best 10 ratios of similarity between the first three form cosmos vs all tops
cnt = 0
ratio_all = []

for address1 in lower_chk01_cosmos:
    cnt += 1
    print ("the currnt count is {}".format(cnt))
    if cnt > 3:
        break
    
     
    curr_sim = difflib.get_close_matches(address1, addr1_all, 10)
    print ("{}. for '{}',there are {} close matches:\n{}".format(cnt, address1, len(curr_sim), curr_sim))
    ratio = []
    sm = difflib.SequenceMatcher()
    sm.set_seq1(address1)
    for s in curr_sim:
        sm.set_seq2(s)
        ratio.append(sm.ratio())
    ratio_all.append(ratio)

In [ ]:
%%time

# Calculate the best 10 ratios of similarity between the first three form cosmos vs all tops
cnt = 0
ratio_all = []

for address1 in lower_chk01_cosmos:
    cnt += 1
    print ("the currnt count is {}".format(cnt))
    #if cnt <= 3:
    #    continue
    if cnt > 2:
        break
         
    curr_sim = difflib.get_close_matches(address1, addr1_all, 10)
    print ("{}. for '{}',there are {} close matches:\n{}".format(cnt, address1, len(curr_sim), curr_sim))
    ratio = []
    sm = difflib.SequenceMatcher()
    sm.set_seq1(address1)
    for s in curr_sim:
        sm.set_seq2(s)
        ratio.append(sm.ratio())
    ratio_all.append(ratio)

https://stackoverflow.com/questions/2363731/append-new-row-to-old-csv-file-python

In [ ]:
# test the append mode to csv file
with open('beneficiary.csv', 'a') as newFile:
    newFileWriter = csv.writer(newFile)




In [ ]:
len(addr1_all)

In [ ]:

cnt = 0
ratio_all = []

for address1 in lower_chk01_cosmos:
    cnt += 1
    print ("the currnt count is {}".format(cnt))
    if cnt > 3:
        break
    
    for 
    curr_sim = difflib.get_close_matches(address1, lower_chk01_tops, 1)
    print ("{}. for '{}',there are {} close matches:\n{}".format(cnt, address1, len(curr_sim), curr_sim))
    ratio = []
    sm = difflib.SequenceMatcher()
    sm.set_seq1(address1)
    for s in curr_sim:
        sm.set_seq2(s)
        ratio.append(sm.ratio())
    ratio_all.append(ratio)

In [ ]:
import psutil
psutil.virtual_memory()

In [ ]:
import tensorflow

In [ ]:
len(ratio_all)

In [ ]:
ratio_all[2]

In [ ]:
# Let us reverse tops with cosmos

cnt = 0
 
for address1 in lower_chk01_tops:
    cnt += 1
    print ("the currnt count is {}".format(cnt))
    if cnt > 3:
        break
    ratio_all = []
    curr_sim = difflib.get_close_matches(address1, lower_chk01_cosmos)
    ratio = []
    sm = difflib.SequenceMatcher()
    sm.set_seq1(address1)
    for s in curr_sim:
        sm.set_seq2(s)
        ratio.append(sm.ratio())
    ratio_all.append(ratio)

In [ ]:
lower_chk01_cosmos.head()

In [ ]:
samp_tops.member_last_name.value_counts(dropna=False).head(10)

In [ ]:
samp_tops.member_first_name.value_counts(dropna=False).head(10)

In [ ]:
fn_counts = samp_tops.member_first_name.value_counts(dropna=False)
addr1_counts = samp_tops.addr_line_1.value_counts(dropna=False)

fn_counts_c = samp_cosmos.member_first_name.value_counts(dropna=False)
addr1_counts_c = samp_cosmos.addr_line_1.value_counts(dropna=False)

In [ ]:
print ("tops has {} different first names, with totally {} 'Mike's. ".format(fn_counts.count(), fn_counts['MIKE']))
print ("tops has {} different address 1 out of {} claims ".format(addr1_counts.count(), samp_tops.member_individual_identifier.count()))

print ("cosmos has {} different first names, with totally {} 'Mike's. ".format(fn_counts_c.count(), fn_counts_c['MIKE']))
print ("cosmos has {} different address 1 out of {} claims ".format(addr1_counts_c.count(), samp_cosmos.member_individual_identifier.count()))

In [ ]:
addr1_01_tops = list(samp_tops['addr_line_1'])
addr1_01_cosmos = list(samp_cosmos['addr_line_1'])

In [ ]:
addr1_set1_tops = set(addr1_01_tops)
addr1_set1_cosmos = set(addr1_01_cosmos)

In [ ]:
print ("the length of the tops list={} and set ={}".format(len(addr1_01_tops), len(addr1_set1_tops)))

In [ ]:
# let us also set up the dataframes by pandas:
df_dedup01_tops = samp_tops.addr_line_1.apply(lambda x:x.lower().translate({None:"'.,"}) if pd.notnull(x) else '')
df_dedup01_cosmos = samp_cosmos.addr_line_1.apply(lambda x:x.lower().translate({None:"'.,"}) if pd.notnull(x) else '')

In [ ]:
addr1_01_tops
MONTE CARLO WAY

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import hashlib

import string
import random

In [ ]:
# N-gram function courtesy of Peter Norvig
def ngrams(seq, n):
    "List all the (overlapping) ngrams in a sequence."
    return [seq[i:i+n] for i in range(1+len(seq)-n)]

def permute(w, p):
    return ''.join(string.ascii_lowercase[p[ord(c) - ord('a')]] for c in w)

def jaccard_similarity(a, b):
    x = set(a)
    y = set(b)
    return len(x & y) / len(x | y)

def m(w, s, p):
    return sorted(permute(w, p))[:s]

def estimator(d1, d2, s, p):
    ma = set(m(d1, s, p))
    mb = set(m(d2, s, p))

    mab = set(sorted(list(ma | mb))[:s])
    return len(mab & ma & mb)/len(mab)

In [ ]:
random.seed(123456)

def rand_hash(p):
    a = random.randint(1,p-1)
    b = random.randint(1,p-1)
    def h(x):
        return ((a * x + b) % p) 
    return h

def min_hash_sig(s):
    return [min(h(ord(c) - ord('a')) for c in s) for h in hash_fam]

In [ ]:
# read member information from the directory
tops_frame = pd.DataFrame()
list_tops = []
for filename in topses:
    if filename[-1] == '0':
        curr_tops = pd.read_csv(os.path.join(path_tops, filename), sep = ',', names = mem_attrs, error_bad_lines=False)
        list_tops.append(curr_tops)
tops_frame = pd.concat(list_tops)

## Analysis for the batch processing
According to the above warning, there may be two posibilities:
1. Different data type issue; then we can try to set dtype = False
2. Memory issue: we should computer similarity chunk by chunk 

In [ ]:
cnt_t = 0
cnt_m = 0

stop_t = 1
stop_m = 1

for file_tops in topses:
    if file_tops[-1] == '0':
        curr_tops = pd.read_csv(os.path.join(path_tops, file_tops), sep = ',', names = mem_attrs, error_bad_lines=False)
        for file_cosmos in cosmoses:
            if file_cosmos[-1] == '0':
                cnt_cosmos += 1
                curr_cosmos = pd.read_csv(os.path.join(path_cosmos, file_cosmos), sep = ',', names = mem_attrs, error_bad_lines=False)
                
                if cnt_cosmos >= stop_cosmos:
                    break;
            if cnt_tops >= stop_tops:
                break;

In [ ]:
samp_tops = pd.read_csv(os.path.join(path_tops, topses[1]), sep = ',', names = mem_attrs, error_bad_lines=False)
samp_cosmos = pd.read_csv(os.path.join(path_cosmos, cosmoses[1]), sep = ',', names = mem_attrs, error_bad_lines=False)

In [ ]:
samp_tops 

In [ ]:
samp_cosmos

In [ ]:
list001_tops = list(samp_tops['addr_line_1'])
list001_cosmos = list(samp_cosmos['addr_line_1'])

In [ ]:
topses

In [ ]:
top001 = pd.read_csv(os.path.join(path_tops, topses[1]), sep = ',', names = mem_attrs, error_bad_lines=False)

In [ ]:
top001.head()

In [ ]:
os.listdir('/mapr/datalake/uhclake/perf/ESLOADER/ymalcow')

In [ ]:
top001.count()

In [ ]:
os.listdir('/mapr/datalake/uhclake/perf/Sqoop_RX/ymalcow/501_mem_match_proj1')

In [ ]:
os.listdir('/mapr/datalake/uhclake/perf/Sqoop_RX/ymalcow/501_mem_match_proj1/tops_all_csv')

In [ ]:
os.listdir('/mapr/datalake/uhclake/perf/ESLOADER')

In [ ]:
print (os.listdir('/mapr'))

In [ ]:
from pyspark import SparkContext;
from pyspark import SparkConf
conf = (SparkConf().setMaster("yarn-client").setAppName("test")
        .set("spark.executor.memory", "10g")
        .set("spark.executor.instances", "11")
        .set("spark.driver.memory", "5g")
        .set("spark.shuffle.service.enabled","true")
        .set("spark.dynamicAllocation.enabled","true")
        .set("spark.dynamicAllocation.minExecutors","1")
        .set("spark.yarn.queue", "fabpwond_q1")
        )

In [ ]:
sparksql

In [ ]:
cnt2018 = spark.sql("select count(*) from h2s.cosmos_professional_claim where mth in ('2018-01','2018-01','2018-01')");
cnt2018.show(1);

In [ ]:
cnt2018 = spark.sql("select count(*) from h2s.cosmos_professional_claim where mth in ('2018-01','2018-01','2018-01')");
cnt2018.show(1);